<a href="https://colab.research.google.com/github/Angel-dash/Abstractive-Text-Summarization/blob/main/DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [1]:
#from google.colab import drive
#drive.mount('/content/drive')


In [2]:
#!unzip /content/drive/MyDrive/MiniProjectdata/data.zip


Archive:  /content/drive/MyDrive/MiniProjectdata/data.zip
  inflating: cnn_dailymail/test.csv  
  inflating: cnn_dailymail/train.csv  
  inflating: cnn_dailymail/validation.csv  


In [9]:
import numpy as np
import pandas as pd
import re
import tensorflow
import nltk
from bs4 import BeautifulSoup
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import warnings
pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore")
import textwrap
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [15]:
data_train=pd.read_csv('./data/test.csv')
data_test=pd.read_csv('./data/test.csv')
data_validation=pd.read_csv('./data/validation.csv')

Data preprocessing


In [16]:
data = pd.concat([data_train, data_test, data_validation ])
df = data

In [19]:
new_df=df.sample(50000,replace=True)

In [20]:
new_df.head(3)

,id,article,highlights
9095,532af494c6d152a7b29deea9d8089ec6f83dc997,"A mother has described how she heard a doctor asking her husband about switching off her life-support as she lay in a medically-induced coma after collapsing from a rare condition. Jenny Bone, who...",Jenny Bone has recovered after husband John refused to end her life .\nMrs Bone fell in to a coma after falling ill with Guillain-Barre Syndrome .\nNerve condition meant she appeared unconsciousne...
7656,691fd676dc40ff633067e8f255c26d6a31b9f631,Thibaut Courtois believes Chelsea's win at West Ham was the sort which showed they can be Premier League champions. Chelsea followed up last Sunday's Capital One Cup final victory over Tottenham w...,Chelsea beat West Ham 1-0 at Upton Park on Wednesday night .\nEden Hazard scored the game's only goal with a first-half header .\nThe Blues retained five-point lead over Manchester City at the top...
3702,656f43d5890e147dbc6fef1d96e033e142bd5a34,Manchester United striker Wayne Rooney has revealed he gets annoyed when fans chant his name at him in the street in an interview with Ant and Dec's 'little' counterparts on Saturday Night Takeawa...,Wayne Rooney is set to appear on Ant and Dec's Saturday Night Takeaway .\nThe England captain jokes around in interview with Little Ant and Dec .\nManchester United forward admits that public atte...


In [21]:
new_df.shape

(50000, 3)

In [22]:
sample=new_df.sample()
print("****Article****")
print(sample.article.values)
print('****Highlights****')
print(sample.highlights.values)

****Article****
['One in three 11-year-olds is still overweight or obese - but an expert has warned that the country’s fat youngsters are now ‘getting fatter’ Britain now has 140,000 ‘super-fat’ children who will need gastric bands as adults if they do not slim down, according to a leading expert. The number of seriously overweight children has soared in recent years as Ministers have failed to tackle the problem, said Professor Paul Gately. Official statistics suggest childhood obesity rates have plateaued in recent years – although one in three 11-year-olds is still overweight or obese. But Prof Gately, who treats obese patients, said the figures disguised the fact that the country’s fat youngsters were now ‘getting fatter’. Speaking at an event to discuss whether people were too worried about childhood obesity, he said the problem was ‘much worse’ than many feared. Overall rates had stabilised, but only because middle-class families had started heeding advice to eat healthily and ta

Checking for Null values


In [23]:
new_df.isna().sum()

id            0
article       0
highlights    0
dtype: int64

Checking for duplicates


In [24]:
new_df.duplicated(subset=['article','highlights']).sum()

29152

In [25]:
new_df=new_df.drop_duplicates(subset=['article','highlights'])
new_df.shape

(20848, 3)

In [26]:
new_df.duplicated(subset=['article','highlights']).sum()

0

In [27]:
new_df['article'][:10]

9095     A mother has described how she heard a doctor asking her husband about switching off her life-support as she lay in a medically-induced coma after collapsing from a rare condition. Jenny Bone, who...
7656     Thibaut Courtois believes Chelsea's win at West Ham was the sort which showed they can be Premier League champions. Chelsea followed up last Sunday's Capital One Cup final victory over Tottenham w...
3702     Manchester United striker Wayne Rooney has revealed he gets annoyed when fans chant his name at him in the street in an interview with Ant and Dec's 'little' counterparts on Saturday Night Takeawa...
1240     A nine-year-old boy found a message in a bottle while exploring a wooded beach of a small lake island on a family vacation - and then realized it was addressed to his grandfather. Nolan Rogers had...
5624     In cities that are as well-known for their burgeoning fashion scene as they are their must-see monuments and ever-evolving culture, it's perhaps no surpris

In [28]:
new_df['article'] = new_df['article'].str.lower()
new_df['highlights'] = new_df['highlights'].str.lower()

new_df.head()

,id,article,highlights
9095,532af494c6d152a7b29deea9d8089ec6f83dc997,"a mother has described how she heard a doctor asking her husband about switching off her life-support as she lay in a medically-induced coma after collapsing from a rare condition. jenny bone, who...",jenny bone has recovered after husband john refused to end her life .\nmrs bone fell in to a coma after falling ill with guillain-barre syndrome .\nnerve condition meant she appeared unconsciousne...
7656,691fd676dc40ff633067e8f255c26d6a31b9f631,thibaut courtois believes chelsea's win at west ham was the sort which showed they can be premier league champions. chelsea followed up last sunday's capital one cup final victory over tottenham w...,chelsea beat west ham 1-0 at upton park on wednesday night .\neden hazard scored the game's only goal with a first-half header .\nthe blues retained five-point lead over manchester city at the top...
3702,656f43d5890e147dbc6fef1d96e033e142bd5a34,manchester united striker wayne rooney has revealed he gets annoyed when fans chant his name at him in the street in an interview with ant and dec's 'little' counterparts on saturday night takeawa...,wayne rooney is set to appear on ant and dec's saturday night takeaway .\nthe england captain jokes around in interview with little ant and dec .\nmanchester united forward admits that public atte...
1240,29bc011843cbdb17759ee679af54d753b2d9a119,a nine-year-old boy found a message in a bottle while exploring a wooded beach of a small lake island on a family vacation - and then realized it was addressed to his grandfather. nolan rogers had...,nolan rogers found the letter while fishing with his family on lake murray in south carolina .\nhis mother quickly recognized the handwriting as her friend diane bryant's .\nbryant wrote the lette...
5624,1f25f86d975211c64655e6e8802651fc6e66a83b,"in cities that are as well-known for their burgeoning fashion scene as they are their must-see monuments and ever-evolving culture, it's perhaps no surprise that hotel staff are as well-dressed as...","hotels around the world are now hiring top designs to craft their uniforms .\nnarcisco rodriguez designed dresses for park hyatt new york hotel staff .\nat toronto's shangri la hotel, sunny fong w..."


In [29]:
new_df['article']=new_df['article'].str.lower()

new_df['highlights']=new_df['highlights'].str.lower()

Converting to lower case


In [30]:
new_df.head()

,id,article,highlights
9095,532af494c6d152a7b29deea9d8089ec6f83dc997,"a mother has described how she heard a doctor asking her husband about switching off her life-support as she lay in a medically-induced coma after collapsing from a rare condition. jenny bone, who...",jenny bone has recovered after husband john refused to end her life .\nmrs bone fell in to a coma after falling ill with guillain-barre syndrome .\nnerve condition meant she appeared unconsciousne...
7656,691fd676dc40ff633067e8f255c26d6a31b9f631,thibaut courtois believes chelsea's win at west ham was the sort which showed they can be premier league champions. chelsea followed up last sunday's capital one cup final victory over tottenham w...,chelsea beat west ham 1-0 at upton park on wednesday night .\neden hazard scored the game's only goal with a first-half header .\nthe blues retained five-point lead over manchester city at the top...
3702,656f43d5890e147dbc6fef1d96e033e142bd5a34,manchester united striker wayne rooney has revealed he gets annoyed when fans chant his name at him in the street in an interview with ant and dec's 'little' counterparts on saturday night takeawa...,wayne rooney is set to appear on ant and dec's saturday night takeaway .\nthe england captain jokes around in interview with little ant and dec .\nmanchester united forward admits that public atte...
1240,29bc011843cbdb17759ee679af54d753b2d9a119,a nine-year-old boy found a message in a bottle while exploring a wooded beach of a small lake island on a family vacation - and then realized it was addressed to his grandfather. nolan rogers had...,nolan rogers found the letter while fishing with his family on lake murray in south carolina .\nhis mother quickly recognized the handwriting as her friend diane bryant's .\nbryant wrote the lette...
5624,1f25f86d975211c64655e6e8802651fc6e66a83b,"in cities that are as well-known for their burgeoning fashion scene as they are their must-see monuments and ever-evolving culture, it's perhaps no surprise that hotel staff are as well-dressed as...","hotels around the world are now hiring top designs to craft their uniforms .\nnarcisco rodriguez designed dresses for park hyatt new york hotel staff .\nat toronto's shangri la hotel, sunny fong w..."


In [31]:
#removing html tags
import re
def remove_html_tags(text):
    if isinstance(text, str):
        pattern = re.compile('<.*?>')
        return pattern.sub(r'', text)
    else:
        return text

In [32]:
new_df['article'] = new_df['article'].apply(remove_html_tags)

new_df['highlights'] = new_df['highlights'].apply(remove_html_tags)

In [33]:
new_df.head()

,id,article,highlights
9095,532af494c6d152a7b29deea9d8089ec6f83dc997,"a mother has described how she heard a doctor asking her husband about switching off her life-support as she lay in a medically-induced coma after collapsing from a rare condition. jenny bone, who...",jenny bone has recovered after husband john refused to end her life .\nmrs bone fell in to a coma after falling ill with guillain-barre syndrome .\nnerve condition meant she appeared unconsciousne...
7656,691fd676dc40ff633067e8f255c26d6a31b9f631,thibaut courtois believes chelsea's win at west ham was the sort which showed they can be premier league champions. chelsea followed up last sunday's capital one cup final victory over tottenham w...,chelsea beat west ham 1-0 at upton park on wednesday night .\neden hazard scored the game's only goal with a first-half header .\nthe blues retained five-point lead over manchester city at the top...
3702,656f43d5890e147dbc6fef1d96e033e142bd5a34,manchester united striker wayne rooney has revealed he gets annoyed when fans chant his name at him in the street in an interview with ant and dec's 'little' counterparts on saturday night takeawa...,wayne rooney is set to appear on ant and dec's saturday night takeaway .\nthe england captain jokes around in interview with little ant and dec .\nmanchester united forward admits that public atte...
1240,29bc011843cbdb17759ee679af54d753b2d9a119,a nine-year-old boy found a message in a bottle while exploring a wooded beach of a small lake island on a family vacation - and then realized it was addressed to his grandfather. nolan rogers had...,nolan rogers found the letter while fishing with his family on lake murray in south carolina .\nhis mother quickly recognized the handwriting as her friend diane bryant's .\nbryant wrote the lette...
5624,1f25f86d975211c64655e6e8802651fc6e66a83b,"in cities that are as well-known for their burgeoning fashion scene as they are their must-see monuments and ever-evolving culture, it's perhaps no surprise that hotel staff are as well-dressed as...","hotels around the world are now hiring top designs to craft their uniforms .\nnarcisco rodriguez designed dresses for park hyatt new york hotel staff .\nat toronto's shangri la hotel, sunny fong w..."


In [34]:
#removing punchuation


import string
string.punctuation
exclude = string.punctuation
def remove_punc(text):
    if isinstance(text, str):
        return ''.join(ch for ch in text if ch not in exclude)
    else:
        return str(text)  # Convert non-string data to string

In [35]:
new_df['article'] = new_df['article'].apply(remove_punc)

new_df['highlights'] = new_df['highlights'].apply(remove_punc)

In [36]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')

print(stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [37]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
new_df['article'] = new_df['article'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
new_df['highlights'] = new_df['highlights'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [38]:
data_train.head(1)

,id,article,highlights
0,92c514c913c0bdfe25341af9fd72b29db544099b,"Ever noticed how plane seats appear to be getting smaller and smaller? With increasing numbers of people taking to the skies, some experts are questioning if having such packed out planes is putti...",Experts question if packed out planes are putting passengers at risk .\nU.S consumer advisory group says minimum space must be stipulated .\nSafety tests conducted on planes with more leg room th...


In [39]:
new_df.head(1)

,id,article,highlights
9095,532af494c6d152a7b29deea9d8089ec6f83dc997,mother described heard doctor asking husband switching lifesupport lay medicallyinduced coma collapsing rare condition jenny bone feared catastrophic brain damage going cardiac arrest lay listenin...,jenny bone recovered husband john refused end life mrs bone fell coma falling ill guillainbarre syndrome nerve condition meant appeared unconsciousness brain dead heard doctors ask husband permiss...


In [27]:
#!pip install contractions


In [40]:
max_len_text=80
max_len_summary=10

In [42]:
from sklearn.model_selection import train_test_split
x_tr,x_val,y_tr,y_val=train_test_split(df['article'],df['highlights'],test_size=0.1,random_state=0,shuffle=True)

In [43]:
#prepare a tokenizer for reviews on training data
x_tokenizer = Tokenizer()
x_tokenizer.fit_on_texts(list(x_tr))

#convert text sequences into integer sequences
x_tr    =   x_tokenizer.texts_to_sequences(x_tr)
x_val   =   x_tokenizer.texts_to_sequences(x_val)

#padding zero upto maximum length
x_tr    =   pad_sequences(x_tr,  maxlen=max_len_text, padding='post')
x_val   =   pad_sequences(x_val, maxlen=max_len_text, padding='post')

x_voc_size   =  len(x_tokenizer.word_index) +1

In [44]:
#preparing a tokenizer for summary on training data
y_tokenizer = Tokenizer()
y_tokenizer.fit_on_texts(list(y_tr))

#convert summary sequences into integer sequences
y_tr    =   y_tokenizer.texts_to_sequences(y_tr)
y_val   =   y_tokenizer.texts_to_sequences(y_val)

#padding zero upto maximum length
y_tr    =   pad_sequences(y_tr, maxlen=max_len_summary, padding='post')
y_val   =   pad_sequences(y_val, maxlen=max_len_summary, padding='post')

y_voc_size  =   len(y_tokenizer.word_index) +1

LSTM model building


In [45]:
from keras import backend as K
K.clear_session()
latent_dim = 500

In [46]:
#encoder
encoder_inputs = Input(shape=(max_len_text,))
enc_emb = Embedding(x_voc_size, latent_dim,trainable=True)(encoder_inputs)

In [47]:
#LSTM 1
encoder_lstm1 = LSTM(latent_dim,return_sequences=True,return_state=True)
encoder_output1, state_h1, state_c1 = encoder_lstm1(enc_emb)

#LSTM2
encoder_lstm2 = LSTM(latent_dim,return_sequences=True,return_state=True)
encoder_output2, state_h2, state_c2 = encoder_lstm2(encoder_output1)

encoder_lstm3=LSTM(latent_dim, return_state=True, return_sequences=True)
encoder_outputs, state_h, state_c= encoder_lstm3(encoder_output2)

In [48]:
#decoder
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(y_voc_size, latent_dim,trainable=True)
dec_emb = dec_emb_layer(decoder_inputs)

In [49]:
#LSTM using encoder_states as initial state
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs,decoder_fwd_state, decoder_back_state = decoder_lstm(dec_emb,initial_state=[state_h, state_c])


In [54]:
#Attention Layer
from src.layers.attention import AttentionLayer
attn_layer = AttentionLayer(name='attention_layer')
attn_out, attn_states = attn_layer([encoder_outputs, decoder_outputs])

In [55]:
# Concat attention output and decoder LSTM output 
decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attn_out])

In [56]:
#Dense layer
decoder_dense = TimeDistributed(Dense(y_voc_size, activation='softmax')) 
decoder_outputs = decoder_dense(decoder_concat_input) 

In [57]:
# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs) 
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 80)]                 0         []                            
                                                                                                  
 embedding (Embedding)       (None, 80, 500)              1006210   ['input_1[0][0]']             
                                                          00                                      
                                                                                                  
 lstm (LSTM)                 [(None, 80, 500),            2002000   ['embedding[0][0]']           
                              (None, 500),                                                        
                              (None, 500)]                                                    

In [58]:
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')

In [59]:
#using Early stoppage to stop the training of the model once the validation increasers

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)

In [60]:
history=model.fit([x_tr,y_tr[:,:-1]], y_tr.reshape(y_tr.shape[0],y_tr.shape[1], 1)[:,1:] ,epochs=50,callbacks=[es],batch_size=512, validation_data=([x_val,y_val[:,:-1]], y_val.reshape(y_val.shape[0],y_val.shape[1], 1)[:,1:]))

Epoch 1/50

 3/64 [>.............................] - ETA: 4:34:07 - loss: 11.0955

In [ ]:
from matplotlib import pyplot 
pyplot.plot(history.history['loss'], label='train') 
pyplot.plot(history.history['val_loss'], label='test') 
pyplot.legend() pyplot.show()

In [ ]:
# encoder inference
encoder_model = Model(inputs=encoder_inputs,outputs=[encoder_outputs, state_h, state_c])

In [ ]:
# decoder inference
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_hidden_state_input = Input(shape=(max_len_text,latent_dim))

In [ ]:

# Get the embeddings of the decoder sequence
dec_emb2= dec_emb_layer(decoder_inputs)

In [ ]:
# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=[decoder_state_input_h, decoder_state_input_c])



In [ ]:
#attention inference
attn_out_inf, attn_states_inf = attn_layer([decoder_hidden_state_input, decoder_outputs2])
decoder_inf_concat = Concatenate(axis=-1, name='concat')([decoder_outputs2, attn_out_inf])

In [ ]:
# A dense softmax layer to generate prob dist. over the target vocabulary
decoder_outputs2 = decoder_dense(decoder_inf_concat)

In [ ]:
# Final decoder model
decoder_model = Model(
[decoder_inputs] + [decoder_hidden_state_input,decoder_state_input_h, decoder_state_input_c],
[decoder_outputs2] + [state_h2, state_c2])

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    e_out, e_h, e_c = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))

    # Chose the 'start' word as the first word of the target sequence
    target_seq[0, 0] = target_word_index['start']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c])

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = reverse_target_word_index[sampled_token_index]

        if(sampled_token!='end'):
            decoded_sentence += ' '+sampled_token

            # Exit condition: either hit max length or find stop word.
            if (sampled_token == 'end' or len(decoded_sentence.split()) >= (max_len_summary-1)):
                stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update internal states
        e_h, e_c = h, c

    return decoded_sentence

In [ ]:
def seq2summary(input_seq):
    newString=''
    for i in input_seq:
      if((i!=0 and i!=target_word_index['start']) and i!=target_word_index['end']):
        newString=newString+reverse_target_word_index[i]+' '
    return newString

def seq2text(input_seq):
    newString=''
    for i in input_seq:
      if(i!=0):
        newString=newString+reverse_source_word_index[i]+' '
    return newString

In [ ]:
for i in range(len(x_val)):
  print("Review:",seq2text(x_val[i]))
  print("Original summary:",seq2summary(y_val[i]))
  print("Predicted summary:",decode_sequence(x_val[i].reshape(1,max_len_text)))
  print("\n")